In [39]:
import pandas as pd
import csv, requests, os, re, itertools, googlemaps, codecs
from tqdm import tqdm
from time import sleep
from fuzzywuzzy import fuzz
import simplejson as json

In [40]:
gmaps = googlemaps.Client(key="")

In [41]:
airports = pd.read_csv("airports.csv", header=None)
airrights = pd.read_csv("flight_rights.tsv", sep="\t")
treaties = pd.read_csv("treaties.tsv", sep="\t")
cities = pd.read_csv('cities_en.tsv', sep='\t')
countries = pd.read_csv('countries_en.tsv', sep='\t')
schedules = pd.read_csv('allschedules.tsv', sep='\t')

In [42]:
airports = airports.loc[:, [2, 3, 4]]
airports.columns = ['encity', 'encountry', 'icao_airport']

In [43]:
repl_city = dict(cities.loc[:, ["ukcity", "encity"]].values.tolist()[0::1])
repl_country = dict(countries.loc[:, ["ukcountry", "encountry"]].values.tolist()[0::1])

### replace cities

In [44]:
treaties['encities_ua'] = treaties['cities_ua']
treaties['encities_foreign'] = treaties['cities_foreign']
airrights['from_encity'] = airrights['from_city']
airrights['to_encity'] = airrights['to_city']

for uk, en in repl_city.items():
    treaties.encities_ua = treaties.encities_ua.str.replace(uk, en)
    treaties.encities_foreign = treaties.encities_foreign.str.replace(uk, en)
    airrights.from_encity = airrights.from_encity.str.replace(uk, en)
    airrights.to_encity = airrights.to_encity.str.replace(uk, en)
    
treaties.encities_ua = treaties.encities_ua.str.replace('всі', 'all')
treaties.encities_foreign = treaties.encities_foreign.str.replace('всі', 'all')

### replace countries

In [45]:
treaties['encountry'] = treaties['country']
airrights['encountry'] = airrights['country']

for uk, en in repl_country.items():
    treaties.encountry = treaties.encountry.str.replace(uk, en)
    airrights.encountry = airrights.encountry.str.replace(uk, en)

treaties.encountry = treaties.encountry.str.replace('[^A-Za-z -]', '').str.strip()
airrights.encountry = airrights.encountry.str.replace('[^A-Za-z -]', '').str.strip()
treaties.encountry = treaties.encountry.str.replace('\s+', ' ').str.strip()
airrights.encountry = airrights.encountry.str.replace('\s+', ' ').str.strip()

## replace icao codes of airports

In [46]:
repl_aircity = dict(airports.loc[:, ["icao_airport", "encity"]].values.tolist()[0::1])
repl_aircountry = dict(airports.loc[:, ["icao_airport", "encountry"]].values.tolist()[0::1])

In [47]:
schedules['encity_from'] = schedules['route_from']
schedules['encity_to'] = schedules['route_to']
schedules['encountry_from'] = schedules['route_from']
schedules['encountry_to'] = schedules['route_to']

for icao, en in repl_aircity.items():
    if isinstance(icao, float) or isinstance(en, float):
        continue
    schedules.encity_from = schedules.encity_from.str.replace(icao, en)
    schedules.encity_to = schedules.encity_to.str.replace(icao, en)
    
for icao, en in repl_aircountry.items():
    if isinstance(icao, float) or isinstance(en, float):
        continue
    schedules.encountry_from = schedules.encountry_from.str.replace(icao, en)
    schedules.encountry_to = schedules.encountry_to.str.replace(icao, en)

In [48]:
schedules = schedules.loc[(pd.notnull(schedules.encity_from) &
                           pd.notnull(schedules.encity_to) &
                           pd.notnull(schedules.encountry_from) &
                           pd.notnull(schedules.encountry_to)), ]

## check if there are duplicated city names

In [49]:
cities = (treaties.encities_ua.values.tolist() +
          treaties.encities_foreign.values.tolist() +
          airrights.from_encity.values.tolist() +
          airrights.to_encity.values.tolist() +
          schedules.encity_from.values.tolist() +
          schedules.encity_to.values.tolist())

cities = [city.split("|") for city in cities
          if city != "all" and not isinstance(city, float)]

cities = list(set([city.strip() for city in itertools.chain.from_iterable(cities)]))

In [50]:
for c1, c2 in itertools.combinations(cities, 2):
    match = fuzz.token_set_ratio(c1, c2)
    if match > 60:
        print(c1, c2, match)

Malaga Palanga 77
Vienna Vinnytsia 67
Venice Nice 80
Gdansk Donetsk 62
Salzburg Hamburg 67
Geneva Yerevan 62
Tirana Ostrava 62
Tirana Tianjin 62
Tirana Tehran 67
Tirana Astana 67
Kostanay Ostrava 67
Kostanay Astana 71
Yerevan Tehran 62
Berlin Beijing 62
Berlin Lublin 67
Berlin Bahrain 62
Aktau Jakarta 67
Aktau Baku 67
Bologna Cologne 71
Bologna Barcelona 62
Lankaran Larnaca 67
Lankaran Ankara 86
Lankaran Jakarta 67
Munich Zurich 67
Belgrade Tselinograd 63
Casablanca Astana 62
Lublin Tallinn 62
Lublin Ljubljana 67
Larnaca Varna 67
Bergamo Burgas 62
Bahrain Tehran 62
Bucharest Budapest 71
Burgas Hamburg 62
Ankara Jakarta 77
Ankara Karachi 62
Pavlodar Palma 62
Pavlodar Zadar 62
Ostrava Astana 62
Tselinograd Kaliningrad 64
Aleppo Salerno 62
Varna Valencia 62
Astana Palanga 62
Palma Almaty 73
Palma Palanga 67
New Delhi Delhi 100
Amman Dammam 73
Kuwait Kutaisi 62


In [51]:
countries = (treaties.encountry.values.tolist() +
             airrights.encountry.values.tolist() +
             schedules.encountry_from.values.tolist() +
             schedules.encountry_to.values.tolist())

countries = list(set(countries))

In [52]:
for c1, c2 in itertools.combinations(countries, 2):
    match = fuzz.token_set_ratio(c1, c2)
    if match > 60:
        print(c1, c2, match)

Slovenia Slovakia 75
Slovenia Estonia 67
Armenia Turkmenistan 63
Armenia Macedonia 62
Ireland Poland 62
Ireland Switzerland 67
Ireland Iceland 86
Ireland Finland 71
Ireland Thailand 67
Ireland Iran 73
Ireland Israel 62
Syria Austria 67
Syria Serbia 73
France Ukraine 62
Latvia Albania 62
Latvia Austria 62
Latvia Croatia 62
Albania Lithuania 62
Albania Malaysia 67
Malta Malaysia 62
Netherlands Switzerland 64
Netherlands Thailand 63
United States United Arab Emirates 67
United States United Kingdom 63
Spain Pakistan 62
Poland Iceland 62
Poland Finland 62
Lithuania Tunisia 62
South Korea South Africa 70
Tajikistan Turkmenistan 64
Tajikistan Pakistan 78
Austria Russia 62
Austria Serbia 62
Switzerland Iceland 67
Uzbekistan Turkmenistan 64
Uzbekistan Pakistan 67
Kazakhstan Pakistan 67
Iraq Iran 75
Iceland Finland 71
Iceland Thailand 67
Iceland Canada 62
Turkmenistan Tunisia 63
Finland Thailand 67
Georgia Serbia 62
Indonesia India 71
Macedonia Romania 62
Macedonia Estonia 62
Romania Croatia 71

In [54]:
#airrights.to_csv('flight_rights_unicities.tsv', sep='\t', index=False, encoding='utf-8')
#treaties.to_csv('treaties_unicities.tsv', sep='\t', index=False, encoding='utf-8')
#schedules.to_csv('allschedules_unicities.tsv', sep='\t', index=False, encoding='utf-8')

In [55]:
print(treaties.columns, '\n',
      airrights.columns, '\n',
      schedules.columns, '\n')

Index(['country', 'stage', 'date', 'iata_season', 'fw_gen', 'fw_route',
       'fw_airline', 'text', 'cities_ua', 'cities_foreign', 'nua_cities',
       'nforeign_cities', 'isfull', 'comments', 'airlines_total',
       'airlines_route', 'encities_ua', 'encities_foreign', 'encountry'],
      dtype='object') 
 Index(['date', 'right_id', 'type_of_right', 'airline_name', 'icao_airline',
       'iata_airline', 'country', 'from_city', 'to_city', 'min_freq',
       'max_freq', 'valid_from', 'valid_till', 'fnnul_date', 'given_by',
       'given_date', 'annul_reason', 'from_encity', 'to_encity', 'encountry'],
      dtype='object') 
 Index(['airline', 'days_week', 'flight', 'route_from', 'route_to',
       'valid_from', 'valid_till', 'encity_from', 'encity_to',
       'encountry_from', 'encountry_to'],
      dtype='object') 



In [56]:
treaties.columns = ['country', 'stage', 'date', 'iata_season', 'fw_gen', 'fw_route',
                    'fw_airline', 'text', 'cities_ua', 'cities_foreign', 'nua_cities',
                    'nforeign_cities', 'notfull', 'comments', 'airlines_total',
                    'airlines_route', 'encities_ua', 'encities_foreign', 'encountry']

airrights.columns = ['date', 'right_id', 'type_of_right', 'airline_name', 'icao_airline',
                     'iata_airline', 'country', 'from_city', 'to_city', 'min_freq',
                     'max_freq', 'valid_from', 'valid_till', 'annul_date', 'given_by',
                     'given_date', 'annul_reason', 'from_encity', 'to_encity', 'encountry']

# Huge json

In [58]:
countries = (treaties.encountry.values.tolist() +
             airrights.encountry.values.tolist() +
             schedules.encountry_from.values.tolist() +
             schedules.encountry_to.values.tolist())

countries = sorted(list(set(countries)), key=str.lower)

In [60]:
huge_json = []

for country in countries:
    treaties_c = treaties.loc[treaties.encountry == country, :]
    airrights_c = airrights.loc[airrights.encountry == country, :]
    schedules_c = schedules.loc[((schedules.encountry_from == country) |
                                 (schedules.encountry_to == country)), :]
    
        
    c_dict = {'encountry': country,
              'treaties': treaties_c.to_dict(orient='records'),
              'flight_rights': airrights_c.to_dict(orient='records'),
              'schedules': schedules_c.to_dict(orient='records')}
    huge_json.append(c_dict)

with codecs.open('avia_all.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(huge_json))

'[{"encountry": "Norway", "treaties": [{"country": "\\u041d\\u043e\\u0440\\u0432\\u0435\\u0433\\u0456\\u044f", "stage": "\\u0447\\u0438\\u043d\\u043d\\u0430", "date": "27.03.2001", "iata_season": "\\u0432\\u0441\\u0456", "fw_gen": 999, "fw_route": 999, "fw_airline": 999, "text": "\\u0431\\u0435\\u0437 \\u043e\\u0431\\u043c\\u0435\\u0436\\u0435\\u043d\\u044c", "cities_ua": "\\u041a\\u0438\\u0457\\u0432|NA|NA", "cities_foreign": "\\u041e\\u0441\\u043b\\u043e|\\u0421\\u0430\\u043d\\u0434\\u0435\\u0444\\u044c\\u043e\\u0440\\u0434|NA", "nua_cities": 3, "nforeign_cities": 3, "isfull": 0, "comments": NaN, "airlines_total": 999, "airlines_route": 999, "encities_ua": "Kiev|NA|NA", "encities_foreign": "Oslo|Sandefjord|NA", "encountry": "Norway"}], "flight_rights": [{"date": "05.04.2016", "right_id": 1406, "type_of_right": "\\u0420\\u0435\\u0433\\u0443\\u043b\\u044f\\u0440\\u043d\\u0456 \\u043c\\u0456\\u0436\\u043d\\u0430\\u0440\\u043e\\u0434\\u043d\\u0456", "airline_name": "\\u042f\\u043d\\u0415

# Unify city names, everything in English

In [ ]:
cities = (treaties.cities_ua.values.tolist() +
          treaties.cities_foreign.values.tolist() +
          airrights.from_city.values.tolist() +
          airrights.to_city.values.tolist())

cities = [city.split("|") for city in cities
          if city != "всі" and not isinstance(city, float)]

cities = [city for city in itertools.chain.from_iterable(cities)]

cities = pd.DataFrame(list(set(cities)), columns=['ukcity'])
cities['encity'] = None
cities['country'] = None

In [ ]:
for i, city in tqdm(cities.iterrows()):
    result = gmaps.geocode(city['ukcity'])
    if len(result) > 0:
        result = [d for d in result[0]['address_components']
                  if 'political' in d['types']]
        cities.loc[i, 'encity'] = result[0]['short_name']
        cities.loc[i, 'country'] = result[-1]['long_name']
    sleep(0.5)

In [ ]:
#cities.to_csv('cities_en.tsv', sep='\t', encoding='utf-8', index=False)

# Get eng names of countries

In [ ]:
countries = (treaties.country.values.tolist() +
             airrights.country.values.tolist())

countries = pd.DataFrame(list(set(countries)), columns=['ukcountry'])
countries['encountry'] = None

In [ ]:
for i, country in tqdm(countries.iterrows()):
    result = gmaps.geocode(country['ukcountry'])
    if len(result) > 0:
        countries.loc[i, 'encountry'] = result[0]['address_components'][0]['long_name']
    sleep(0.5)

In [ ]:
#countries.to_csv('countries_en.tsv', sep='\t', encoding='utf-8', index=False)

manually check cities